# ---Day 7 : Amplification Circuit---

## ---Part One---

Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of <a src="https://en.wikipedia.org/wiki/Amplifier"> amplifiers </a> already installed on the ship.

There are five amplifiers connected in series; each one receives an input signal and produces an output signal. They are connected such that the first amplifier's output leads to the second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on. The first amplifier's input value is 0, and the last amplifier's output leads to your ship's thrusters.
```
    O-------O  O-------O  O-------O  O-------O  O-------O
0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O
```
The Elves have sent you some ***Amplifier Controller Software*** (your puzzle input), a program that should run on your existing Intcode computer (day5 solution). Each amplifier will need to run a copy of the program.

When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current ***phase setting*** (an integer from 0 to 4). Each phase setting is used ***exactly once***, but the Elves can't remember which amplifier needs which phase setting.

The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits until one arrives.)

Your job is to ***find the largest output signal that can be sent to the thrusters*** by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.

**For example, suppose you want to try the phase setting sequence ```3,1,2,4,0```:**
> Setting amplifier ```A``` to phase setting ```3```, amplifier ```B``` to setting ```1```, ```C``` to ```2```, ```D``` to ```4```, and ```E``` to ```0```.

**Then, you could determine the output signal that gets sent from amplifier ```E``` to the thrusters with the following steps:**

1. Start the copy of the amplifier controller software that will run on amplifier ```A```:
>* At its first input instruction, provide it the amplifier's phase setting, ```3```.
>* At its second input instruction, provide it the input signal, ```0```. 
>* After some calculations, it will use an output instruction to indicate the amplifier's output signal.

2. Start the software for amplifier ```B```:
>* Provide it the phase setting (```1```) and then whatever output signal was produced from amplifier ```A```. 
>* It will then produce a new output signal destined for amplifier ```C```.

3. Start the software for amplifier ```C```:
>* Provide the phase setting (```2```) and the input value from amplifier ```B```
>* Then collect its output signal.

4. Run amplifier ```D```'s software:
>* Provide the phase setting (```4```) and input value from ```C```
>* Collect its output signal.

5. Run amplifier ```E```'s software:
>* Provide the phase setting (```0```) and input value from ```D```
>* Collect its output signal.


6. The final output signal from amplifier ```E``` would be sent to the thrusters. However, this phase setting sequence may not have been the best one; another sequence might have sent a higher signal to the thrusters.

**Try every combination of phase settings on the amplifiers. What is the highest signal that can be sent to the thrusters?**

In [1]:
from itertools import permutations #used for phase setting configurations

### Day 5 Intcode Class, renamed Computer
For some reason my day5 Intcode class wasn't working on my day7 programs so I made the following changes:

* Changed parameters function not to override last parameter with mode 0 (position mode)
* Changed add, multiply, less_than, is_equal functions to keep last parameter in mode 0 (position mode)

In [2]:
class Computer:
    def __init__(self, program):
        self.program = program
        self.pointer = 0
        self.instruction = 0
        self.opcode = { 1 : self.add,
                 2 : self.multiply,
                 3 : self.save,
                 4 : self.output,
                 5 : self.jump_true,
                 6 : self.jump_false,
                 7 : self.less_than,
                 8 : self.is_equal,
                 99 : self.halt }
        
    def get_opcode(self):
        return self.instruction % 100
    
    def get_modes(self):
        return self.instruction // 100
    
    def set_instruction(self):
        self.instruction = self.program[self.pointer]
        self.pointer += 1

    def parameters(self, modes, num_parameters=3):
        mode = [ (modes // 10**n % 10) for n in range(num_parameters) ]
        parameters = [  self.program[i] if mode[i-self.pointer] else self.program[self.program[i]] \
                      for i in range(self.pointer, self.pointer+num_parameters) ]
        
        return parameters

    def add(self, step=3):
        values = self.parameters(self.get_modes())
        # Write over last parameter because it should always be an value indicating the location
        values[-1] = self.program[self.pointer+step-1]
        self.program[values[2]] = values[0] + values[1]
        self.pointer += step
        
    def multiply(self, step=3):
        values = self.parameters(self.get_modes())
        # Write over last parameter because it should always be an value indicating the location
        values[-1] = self.program[self.pointer+step-1]
        self.program[values[2]] = values[0] * values[1]
        self.pointer += step
        
    def save(self, step=1):
        new = input("Input : ")
        self.program[self.program[self.pointer]] = int(new)
        self.pointer += step
        
    def output(self, step=2):
        mode = self.get_modes()
        print( f'Output: {(self.program[self.program[self.pointer]], self.program[self.pointer])[mode]}' )
        self.pointer += step
        
    def jump_true(self, step=2):
        values = self.parameters(self.get_modes(), 2)
        if values[0] != 0:
            self.pointer = values[1]
        else:
            self.pointer += step
            
    def jump_false(self, step=1):
        values = self.parameters(self.get_modes(), 2)
        if values[0] == 0:
            self.pointer = values[1]
        else:
            self.pointer += step
            
    def less_than(self, step=3):
        values = self.parameters(self.get_modes())
        # Write over last parameter because it should always be an value indicating the location
        values[-1] = self.program[self.pointer+step-1]
        self.program[values[2]] = int(values[0] < values[1])
        
        self.pointer += step
    
    def is_equal(self, step=3):
        values = self.parameters(self.get_modes())
        # Write over last parameter because it should always be an value indicating the location
        values[-1] = self.program[self.pointer+step-1]
        self.program[values[2]] = int(values[0]==values[1])
        
        self.pointer += step
        
    def halt(self):
        print("Program Halted.")
        self.instruction = -1
    
    def run(self):
        while self.instruction != -1:
            self.set_instruction()
            # print(self.pointer, self.instruction) # Used for Testing
            self.opcode[self.get_opcode()]()
            # print(self.program[:self.pointer+1], self.program[self.pointer:self.pointer+4]) # Used for Testing

In [3]:
class Amp(Computer):
    def __init__(self, program, input1, input2):
        Computer.__init__(self, program)
        self.temp1 = input1
        self.temp2 = input2
        self.out = 0
    
    def save(self, step=1):
        self.program[self.program[self.pointer]] = int(self.temp1)
        self.temp1 = self.temp2
        self.pointer += step
    
    def output(self, step=1):
        mode = self.get_modes()
        self.out = (self.program[self.program[self.pointer]], self.program[self.pointer])[mode]
        self.pointer += step
        
    def halt(self):
        self.instruction = -1
        
    def run(self):
        while self.instruction != -1:
            self.set_instruction()
            self.opcode[self.get_opcode()]()
        return self.out

In [4]:
PHASE_SETTINGS = list(permutations('01234', 5)) #all permutations of 01234

In [5]:
def max_thrusters(program, settings):
    max_thruster = 0
    phase_seq = []
    for setting in settings:
        signal = 0
        for phase in setting:
            amp = Amp(list(program), phase, signal)
            signal = amp.run()
        if signal > max_thruster : max_thruster, phase_seq = signal, setting
    return max_thruster, phase_seq

In [6]:
#Max thruster signal 43210 (from phase setting sequence 4,3,2,1,0):
ex1 = [ 3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0 ]
max_thrusters(ex1, PHASE_SETTINGS)

(43210, ('4', '3', '2', '1', '0'))

In [7]:
#Max thruster signal 54321 (from phase setting sequence 0,1,2,3,4):
ex2 = [ 3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0 ]
max_thrusters(ex2, PHASE_SETTINGS)

(54321, ('0', '1', '2', '3', '4'))

In [8]:
#Max thruster signal 65210 (from phase setting sequence 1,0,4,3,2):
ex3 = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
max_thrusters(ex3, PHASE_SETTINGS)

(65210, ('1', '0', '4', '3', '2'))

In [9]:
with open('day7.txt') as f:
    program = [ int(i) for i in f.read().split(',') ]
    print(max_thrusters(program, PHASE_SETTINGS))

(67023, ('2', '3', '1', '0', '4'))


## ---Part Two---

It's no good - in this configuration, the amplifiers can't generate a large enough output signal to produce the thrust you'll need. The Elves quickly talk you through rewiring the amplifiers into a ***feedback loop***:

```
      O-------O  O-------O  O-------O  O-------O  O-------O
0 -+->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-.
   |  O-------O  O-------O  O-------O  O-------O  O-------O |
   |                                                        |
   '--------------------------------------------------------+
                                                            |
                                                            v
                                                     (to thrusters)
```

Most of the amplifiers are connected as they were before; amplifier `A`'s output is connected to amplifier `B`'s input, and so on. ***However***, the output from amplifier `E` is now connected into amplifier `A`'s input. This creates the feedback loop: the signal will be sent through the amplifiers ***many times***.

In feedback loop mode, the amplifiers need ***totally different phase settings***: integers from `5` to `9`, again each used exactly once. These settings will cause the Amplifier Controller Software to repeatedly take input and produce output many times before halting. Provide each amplifier its phase setting at its first input instruction; all further input/output instructions are for signals.

Don't restart the Amplifier Controller Software on any amplifier during this process. Each one should continue receiving and sending signals until it halts.

All signals sent or received in this process will be between pairs of amplifiers except the very first signal and the very last signal. To start the process, a `0` signal is sent to amplifier `A`'s input ***exactly once***.

Eventually, the software on the amplifiers will halt after they have processed the final loop. When this happens, the last output signal from amplifier `E` is sent to the thrusters. Your job is to ***find the largest output signal that can be sent to the thrusters*** using the new phase settings and feedback loop arrangement.

**Try every combination of the new phase settings on the amplifier feedback loop. What is the highest signal that can be sent to the thrusters?**

In [10]:
class Amp2(Computer):
    def __init__(self, program, phase, signal):
        Computer.__init__(self, program)
        self.phase = phase
        self.signal = signal
        self.first_run = True
        self.out = 0
        self.halt = False        
    
    def save(self, step=1):
        if self.first_run:
            self.program[self.program[self.pointer]] = int(self.phase)
            self.first_run = False
        else:
            self.program[self.program[self.pointer]] = int(self.signal)
        self.pointer += step
    
    def output(self, step=1):
        mode = self.get_modes()
        self.out = (self.program[self.program[self.pointer]], self.program[self.pointer])[mode]
        self.pointer += step
        self.instruction = -1 #Amps only produce ONE output at a time
        
    def halt(self):
        self.instruction = -1
        self.halt = True
        
    def initialize(self):
        self.instruction = 0
        
    def run(self):
        if not self.first_run or self.halt:
            self.initialize()
        while self.instruction != -1:
            self.set_instruction()
            self.opcode[self.get_opcode()]()
        return self.out

**Checking new class outputs with ex1 from part 1**

In [11]:
ex1 = [ 3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0 ]
ampA = Amp2(ex1,4,0)
ampB = Amp2(ex1,3,ampA.run())
ampC = Amp2(ex1,2,ampB.run())
ampD = Amp2(ex1,1,ampC.run())
ampE = Amp2(ex1,0,ampD.run())
ampE.run()
ampA.out, ampB.out, ampC.out, ampD.out, ampE.out

(4, 43, 432, 4321, 43210)

In [12]:
FEEDBACK_SETTINGS = list(permutations('56789', 5)) #all permutations of 56789

In [13]:
def run_feedback(program, phase_setting, signal=0):
    amps = [ Amp2(list(program), i, 0) for i in phase_setting ]
    amp_num = 0
    while not amps[-1].halt:
        amps[(amp_num+1)%len(amps)].signal = amps[amp_num].run()
        amp_num = (amp_num+1)%len(amps)
    return amps[-1].out

In [14]:
#Max thruster signal 139629729 (from phase setting sequence 9,8,7,6,5):
feed_ex1 = [ 3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5 ]

print(f"Thruster signal: {run_feedback(feed_ex1,'98765')}")

Thruster signal: 139629729


In [15]:
#Max thruster signal 18216 (from phase setting sequence 9,7,8,5,6):

feed_ex2 = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54, \
            -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4, \
            53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]

print(f"Thruster signal: {run_feedback(feed_ex2,'97856')}")

Thruster signal: 18216


In [16]:
def get_max_thrust(program, phase_settings, signal=0, get_phase=False):
    if get_phase:
        thrusts = [ run_feedback(program, phase, signal) for phase in phase_settings ]
        max_thrust = max(thrusts)
        return max_thrust, phase_settings[thrusts.index(max_thrust)]
    return max([ run_feedback(program, phase, signal) for phase in phase_settings ])

In [17]:
print(f'feed_ex1: {get_max_thrust(feed_ex1,FEEDBACK_SETTINGS, get_phase=True)}')
print(f'feed_ex2: {get_max_thrust(feed_ex2,FEEDBACK_SETTINGS)}')

feed_ex1: (139629729, ('9', '8', '7', '6', '5'))
feed_ex2: 18216


In [18]:
with open('day7.txt') as f:
    program = [ int(i) for i in f.read().split(',') ]
    print(get_max_thrust(program, FEEDBACK_SETTINGS,get_phase=True))

(7818398, ('5', '8', '7', '9', '6'))
